<a href="https://colab.research.google.com/github/JC1201/-Intelligent-Edge-Caching-Using-Federated-Learning-to-Predict-Content-Popularity/blob/main/TYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy
from tensorflow.python.ops.gen_dataset_ops import shuffle_dataset

dataset, info = tfds.load("movielens/100k-ratings", split=["train"], with_info=True)

data_set = dataset[0]

def preprocess_data(dataset):

    user_id = tf.strings.to_number(dataset["user_id"], out_type=tf.int32)
    movie_id = tf.strings.to_number(dataset["movie_id"], out_type=tf.int32)

    #casting user rating into 0 and 1 (0 means didn't rate this movie, 1 means did rate this movie)
    did_rate = tf.cast(dataset["user_rating"] > 0, tf.int64)

    features = {
        "user_id": user_id,
        "movie_id": movie_id,
        "user_rating":  did_rate

    }
    return features

data_set = data_set.map(preprocess_data)
dataset_size = tf.data.experimental.cardinality(data_set).numpy()
print(f"Total dataset size: {dataset_size}")

num_client = 5
userS_dataset = []

for i in range(num_client):

    seed = numpy.random.seed(i)
    data_set = data_set.shuffle(buffer_size=dataset_size, seed= seed)
    userS_dataset.append(data_set)

data_set_1 =  data_set.shuffle(buffer_size=dataset_size, seed=42)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/incomplete.CDCVGN_0.1.1/movielens-train.tfrecord*..…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.
Total dataset size: 100000


In [2]:
#size of training data and testing data
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

#extract training data and testing data
train_data_1 = data_set_1.take(int(train_size))
test_data_1 = data_set_1.skip(int(train_size))

train_data = []
test_data = []

for i in range(num_client):

    train_data.append(userS_dataset[i].take(int(train_size)))
    test_data.append(userS_dataset[i].skip(int(train_size)))


In [3]:
userIDs = []
movieIDs = []
ratings = []

unique_user_movie_pair = set()

for example in data_set:

    # Convert TensorFlow tensor to a NumPy value
    user_id = example["user_id"].numpy()
    movie_id = example["movie_id"].numpy()
    rating = example["user_rating"].numpy()

    # Add movie pair only if it is uniuqe
    if (user_id, movie_id) not in unique_user_movie_pair:
        userIDs.append(user_id)
        movieIDs.append(movie_id)
        ratings.append(rating)
        unique_user_movie_pair.add((user_id, movie_id))

# Convert lists to NumPy arrays for later processing
userIDs = numpy.array(userIDs)
movieIDs = numpy.array(movieIDs)
ratings = numpy.array(ratings)

#size of user and movie in training data
num_user = max(userIDs) + 1
num_movie = max(movieIDs) + 1
num_rating = max(ratings) + 1

print(f"Number of unique users: {num_user}")
print(f"Number of unique movies: {num_movie}")


Number of unique users: 944
Number of unique movies: 1683


In [4]:
# Get unique user and movie IDs
unique_userIDs = numpy.unique(userIDs)
unique_movieIDs = numpy.unique(movieIDs)

# Convert moive index into a seqeunce to number

# Create mapping dictionaries
user_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_userIDs)}
movie_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_movieIDs)}

# Map the IDs
userIDs_mapped = numpy.array([user_id_mapping[uid] for uid in userIDs])
movieIDs_mapped = numpy.array([movie_id_mapping[mid] for mid in movieIDs])

# New dimensions
num_user = len(unique_userIDs)
num_movie = len(unique_movieIDs)

print(f"Number of users: {num_user}, Number of movies: {num_movie}")


Number of users: 943, Number of movies: 1682


In [11]:
def integrate_feature_into_matrix(userIDs, movieIDs, ratings, num_user, num_movie):

    # Create a 2D matrix filled with zeros
    user_movie_matrix = numpy.zeros((num_user, num_movie), dtype=numpy.int32)

    # Populate the matrix
    for userID, movieID, rating in zip(userIDs, movieIDs, ratings):
        user_movie_matrix[int(userID), int(movieID)] = rating

    return user_movie_matrix

data_set_matrix = integrate_feature_into_matrix(userIDs_mapped, movieIDs_mapped, ratings, num_user, num_movie)
# train_matrix = data_set_matrix.reshape((1, num_user, num_movie, 1))
# print(train_matrix.shape)

train_percentage = 0.8
train_size = int(num_user * train_percentage)  # Number of users for training
test_size = num_user - train_size

# print(f"Number of users for training: {train_size}")
# print(f"Number of users for testing: {test_size}")

train_matrix = data_set_matrix[:train_size, :]
test_matrix = data_set_matrix[train_size:, :]

# train_matrix = integrate_feature_into_matrix(userIDs_mapped, movieIDs_mapped, ratings, num_user, num_movie)
train_matrix = train_matrix.reshape((1, train_size, num_movie, 1))
test_matrix = test_matrix.reshape((1, test_size, num_movie, 1))
print(train_matrix.shape)
print(test_matrix.shape)


(1, 754, 1682, 1)
(1, 189, 1682, 1)


In [13]:
def build_cnn_model(num_user, num_movie):

    input_matrix = tf.keras.layers.Input(shape=(num_user, num_movie, 1),batch_size = 32, name="user_id")

    cnn_layer = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_matrix)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(4, 4))(cnn_layer) # Added MaxPooling

    cnn_layer = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(cnn_layer)
    cnn_layer = tf.keras.layers.MaxPooling2D(pool_size=(4, 4))(cnn_layer) # Added MaxPooling

    cnn_layer = tf.keras.layers.Flatten()(cnn_layer)

    dense_layer = tf.keras.layers.Dense(128, activation='relu')(cnn_layer)

    output = tf.keras.layers.Dense(num_movie, activation='sigmoid', name="movie_scores")(dense_layer)
    model = tf.keras.models.Model(inputs=input_matrix, outputs=output, name="MoviePopularityModel")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',  # For regression tasks
        metrics=['accuracy']  # Mean Absolute Error for monitoring
    )

    return model


model = build_cnn_model(train_size, num_movie)

model.fit(
    train_matrix,
    train_matrix,  # For self-supervised learning
    epochs=10,
    batch_size=32
)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0000e+00 - loss: 0.2505
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 0.3159
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0000e+00 - loss: 0.2710
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.0000e+00 - loss: 0.2381
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.0000e+00 - loss: 0.2300
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.0000e+00 - loss: 0.2209
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.0000e+00 - loss: 0.2082
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0000e+00 - loss: 0.1943
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.0000e+00 - loss: 0.1801
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.0000e+00 - loss: 0.1660


In [14]:
# Predict movie scores
predictions = model.predict(train_matrix)

# Aggregate scores across users
movie_scores = predictions.mean(axis=0)

# Get top 50 movie indices
# Top 50 in descending ordere
movie_score = numpy.argsort(movie_scores)[:][::-1]

print("Predicted Movies Ranking(by index):")
for rank, movie_idx in enumerate(movie_score, 1):
    print(f"{rank}: Movie Index {movie_idx}, Score {movie_scores[movie_idx] } ")




1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Movies Ranking(by index):
1: Movie Index 514, Score 0.9465681314468384 
2: Movie Index 1304, Score 0.9231312274932861 
3: Movie Index 1242, Score 0.9220446348190308 
4: Movie Index 566, Score 0.9054845571517944 
5: Movie Index 299, Score 0.9015874862670898 
6: Movie Index 791, Score 0.8950477838516235 
7: Movie Index 728, Score 0.8875188231468201 
8: Movie Index 1308, Score 0.8847433924674988 
9: Movie Index 377, Score 0.8801583647727966 
10: Movie Index 1114, Score 0.8798238039016724 
11: Movie Index 1024, Score 0.8777055740356445 
12: Movie Index 97, Score 0.8769144415855408 
13: Movie Index 1632, Score 0.8759111166000366 
14: Movie Index 331, Score 0.8732872009277344 
15: Movie Index 1477, Score 0.861920177936554 
16: Movie Index 819, Score 0.8595210313796997 
17: Movie Index 134, Score 0.8508879542350769 
18: Movie Index 1552, Score 0.8499742746353149 
19: Movie Index 1174, Score 0.8471625447273254 
20: Movie Index 503, Score 0.83807384

In [15]:
# loss, mae = model.evaluate(train_matrix, test_matrix)  # For self-supervised learning, use test_matrix as both inputs and targets
# print(f"Test Loss: {loss}")
# print(f"Test MAE: {mae}")

# predicted_scores = model.predict(test_matrix)
# print(f"Score: {predicted_scores}")


In [65]:
def calculate_caching_hit_rate(predicted_scores, test_matrix, cache_size):

    """
    Calculate the caching hit rate.

    :param predicted_scores: Array of predicted scores for all movies (shape: num_movies).
    :param test_data: Test dataset containing actual movie requests (e.g., movieIDs).
    :param cache_size: Number of movies to cache (top-k based on predicted scores).
    :return: Hit rate as a float.
    """

    predicted_scores = predicted_scores.flatten()

    # Get top-k movies based on predicted scores
    top_k_movies = numpy.argsort(predicted_scores)[-cache_size:][::-1]

    print(top_k_movies)

    #Extract actual requested movie IDs from test_matrix
    requested_movie_ids = set()
    hits = 0
    _,num_user, num_movie,_ = test_matrix.shape

    for user_id in range(num_user):

        user_rating = test_matrix[0, user_id, :, 0]

        requested_movies = numpy.where(user_rating > 0)[0] # Get indices of requested movies
        requested_movie_ids.update(requested_movies)

    print(requested_movie_ids)
    print(set(requested_movie_ids))


    # Convert the set to a list

    # Calculate hit rate
    for movie_id in requested_movie_ids:
        if movie_id in top_k_movies:
            hits += 1

    total_unique_movies = len(requested_movie_ids)
    print(f"Total unique movies: {total_unique_movies}, total hit: {hits}")

    hit_rate = hits / cache_size

    return hit_rate

# Example usage
cache_size = 200  # the number of cached top N movies
hit_rate = calculate_caching_hit_rate(predictions, test_matrix, cache_size)
print(f"Caching Hit Rate: {hit_rate:.2%}")

[ 514 1304 1242  566  299  791  728 1308  377 1114 1024   97 1632  331
 1477  819  134 1552 1174  503  905  624 1390 1614  467 1564  354 1569
  405  229 1652 1217 1153 1084   45  827 1615  642 1307  441  106 1207
  766  297 1274  682  710  660 1096 1159 1237  608  838 1457  327   30
  295  981  892 1194  721 1106  307  485 1537 1195 1605 1515 1267  570
   13  658  312 1182 1116  914  749  803  657 1350 1555  844  971 1351
  144 1543  649 1482 1657 1459   20  450   42  953  186 1464 1172  681
  381 1010  743 1539 1567 1171  232 1186 1146 1420 1275  197  873 1365
  604 1491 1089 1201 1102 1100 1169 1644 1073  286 1454 1016 1301  446
  527 1083 1638 1263 1361  401  414 1317  865 1524 1372 1467  224  585
 1577  243  447 1047 1271 1578  701 1149  436  438 1673  371 1197  400
 1339 1492 1385  121  530 1508  984 1238  151  372 1005 1573  233  670
 1527  116  219 1546 1341 1488  305  663 1346  916 1193  109 1607 1400
  706  164  145 1506 1110  287  169  878  264 1571 1220  970 1501  534
 1211 

What to do next

IF the cach hit rate is high enough

try with multiple user

In [62]:
first_array = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 592, 594, 595, 596, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 666, 668, 669, 670, 671, 672, 673, 674, 675, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 777, 778, 779, 780, 781, 782, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 804, 805, 806, 807, 808, 809, 810, 811, 812, 814, 815, 817, 818, 819, 820, 822, 823, 824, 825, 826, 827, 828, 830, 831, 832, 833, 834, 835, 836, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 853, 854, 855, 858, 859, 860, 861, 862, 863, 864, 865, 866, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 891, 892, 893, 894, 895, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 911, 914, 915, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 954, 955, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1075, 1076, 1077, 1078, 1079, 1081, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1104, 1106, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1123, 1125, 1126, 1128, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1141, 1142, 1143, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1156, 1158, 1159, 1160, 1162, 1163, 1164, 1166, 1167, 1169, 1170, 1171, 1172, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1186, 1187, 1189, 1190, 1192, 1193, 1194, 1195, 1196, 1198, 1199, 1201, 1202, 1203, 1204, 1205, 1207, 1208, 1209, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1233, 1236, 1237, 1238, 1239, 1240, 1242, 1243, 1244, 1245, 1247, 1248, 1250, 1251, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1261, 1264, 1265, 1266, 1267, 1268, 1269, 1271, 1272, 1273, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1290, 1291, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1301, 1302, 1304, 1310, 1311, 1312, 1313, 1314, 1316, 1321, 1322, 1323, 1325, 1329, 1334, 1336, 1343, 1344, 1350, 1352, 1354, 1364, 1366, 1367, 1369, 1374, 1375, 1376, 1377, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1392, 1393, 1394, 1396, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1408, 1410, 1411, 1412, 1414, 1416, 1418, 1420, 1421, 1422, 1424, 1426, 1427, 1429, 1430, 1432, 1433, 1434, 1436, 1438, 1443, 1445, 1447, 1448, 1450, 1453, 1455, 1458, 1461, 1462, 1465, 1466, 1467, 1468, 1470, 1472, 1473, 1474, 1476, 1477, 1478, 1479, 1482, 1486, 1487, 1488, 1495, 1499, 1500, 1502, 1508, 1510, 1511, 1512, 1513, 1516, 1517, 1521, 1526, 1527, 1528, 1529, 1530, 1533, 1536, 1537, 1538, 1539, 1543, 1544, 1552, 1554, 1557, 1559, 1572, 1587, 1588, 1589, 1590, 1591, 1596, 1597, 1599, 1601, 1604, 1606, 1607, 1608, 1609, 1610, 1611, 1614, 1619, 1621, 1627, 1628, 1638, 1642, 1643, 1645, 1651, 1655, 1657, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681}


second_array = [ 514, 1304, 1242,  566,  299,  791,  728, 1308,  377, 1114, 1024,   97, 1632,  331,
 1477,  819,  134, 1552, 1174,  503,  905,  624, 1390, 1614,  467, 1564,  354, 1569,
  405,  229, 1652, 1217, 1153, 1084,   45,  827, 1615,  642, 1307,  441,  106, 1207,
  766,  297, 1274,  682,  710,  660, 1096, 1159]

common_indices = [i for i in second_array if i in first_array]
count_common = len(common_indices)
print(count_common)

40
